In [ ]:
import numpy as np
from pyvo.dal.adhoc import DatalinkResults

import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth

from lsst.rsp import get_tap_service

from pyvo.dal.adhoc import DatalinkResults, SodaQuery
import lsst.geom as geom
from astropy import units as u
from astropy.coordinates import Angle
from astropy.time import Time
import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
from lsst.afw.fits import MemFileManager
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd


In [ ]:
service = get_tap_service("tap")
assert service is not None

Okay let's pick a SSObject ID sort of at random let's get 4 and we'll use one of them

In [ ]:
query = "SELECT * from dp1.SSObject LIMIT 4"


In [ ]:


job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()


In [ ]:
result = job.fetch_result()
result_df = pd.DataFrame(result)

In [ ]:
result_df

Let's pick the last one in the table above and get the visit ID for one an observation of it

In [ ]:
query = f"SELECT * from dp1.DiaSource where  ssObjectId={result_df.ssObjectId[3]} limit 1"
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

result = job.fetch_result()
result_df = pd.DataFrame(result)   

In [ ]:
result_df

In [ ]:
result_df.visit[0], result_df.trailRa[0], result_df.trailDec[0], result_df.band[0]

Now let's setup up some variables we'll use to grab some cutouts

In [ ]:
target_ra = result_df.trailRa[0]
target_dec = result_df.trailDec[0]
visitid = result_df.visit[0]
visitband=result_df.band[0]
circle = (target_ra, target_dec, 0.05)

Starting with displaying in matplotlib

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
afw_display = afwDisplay.Display(frame=1)

Now use ivoa Obscore data to get the specific part of the Rubin image we want to make a cutout of

In [ ]:

query = """SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit={} and CONTAINS(POINT('ICRS', {},{}), s_region) = 1
        """.format(visitid, target_ra, target_dec)
print(query)


In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()
print(len(results))

In [ ]:
results

Let's grab the image data and make the cutout

In [ ]:
datalink_url = results['access_url'][0]
dl_result = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')

In [ ]:

sq = SodaQuery.from_resource(dl_result,
                             dl_result.get_adhocservice_by_id("cutout-sync"),
                             session=get_pyvo_auth())


In [ ]:


spherePoint = geom.SpherePoint(target_ra*geom.degrees, target_dec*geom.degrees)
Radius = 0.01 * u.deg
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             Radius)


In [ ]:


cutout_bytes = sq.execute_stream().read()
sq.raise_if_error()
mem = MemFileManager(len(cutout_bytes))
mem.setData(cutout_bytes, len(cutout_bytes))
exposure = ExposureF(mem)


Display the cutout

In [ ]:


display = afwDisplay.Display()
display.scale('asinh', 'zscale')
display.image(exposure.image)
plt.show()


In [ ]:
afw_display.setMaskTransparency(100)

Now display the cutout in the Firefly viewer

In [ ]:


afwDisplay.setDefaultBackend("firefly")
display = afwDisplay.Display(frame=1)
display.scale('asinh', 'zscale')
display.image(exposure.image)
plt.show()


Let's do this again but to find another observation where the object is somewhere else so we can show it is moving

In [ ]:

query = """SELECT * FROM ivoa.ObsCore
        WHERE dataproduct_subtype = 'lsst.visit_image' and lsst_visit!={} and lsst_band='{}' and CONTAINS(POINT('ICRS', {},{}), s_region) = 1
        """.format(visitid, visitband, target_ra, target_dec)
print(query)

We're asking to display this cutout in a second frame in the firefly viewer

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()
print(len(results))

datalink_url = results['access_url'][0]
dl_result = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())
image_url = dl_result.getrecord(0).get('access_url')

sq = SodaQuery.from_resource(dl_result,
                             dl_result.get_adhocservice_by_id("cutout-sync"),
                             session=get_pyvo_auth())



spherePoint = geom.SpherePoint(target_ra*geom.degrees, target_dec*geom.degrees)
Radius = 0.01 * u.deg
sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
             spherePoint.getDec().asDegrees() * u.deg,
             Radius)


cutout_bytes = sq.execute_stream().read()
sq.raise_if_error()
mem = MemFileManager(len(cutout_bytes))
mem.setData(cutout_bytes, len(cutout_bytes))
exposure = ExposureF(mem)

display = afwDisplay.Display(frame=2)

display.scale('asinh', 'zscale')
display.image(exposure.image)
plt.show()


In [ ]:
results